In [3]:
import pandas as pd
import numpy as np
import os,sys
import operator
from sklearn import model_selection
import string
import nltk
from nltk.corpus import stopwords

In [4]:
words=[]
target=[]
for category in os.listdir(r"C:\Users\sameer\Desktop\20_newsgroups"):
    for document in os.listdir(r'C:\Users\sameer\Desktop\20_newsgroups/'+category):
        with open(r"C:\Users\sameer\Desktop\20_newsgroups/"+category+'/'+document) as file:
            words.append((document,file.read()))
            target.append(category)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\sameer\\Desktop\\20_newsgroups'

In [ ]:
dictionary={}
xtrain,xtest,ytrain,ytest=model_selection.train_test_split(words,target,test_size=0.25,random_state=0)
from nltk.tokenize import word_tokenize       
stop_words = set(stopwords.words('english')) #importing stopwords 
print((xtrain))
for i in xtrain:
    word_tokens = i[1].split()  # the zeroth index contains the folder number and first index actual text
    filtered_sentence = [w.lower() for w in word_tokens if not w.lower() in stop_words ] # removing the stopwords
    for i in range(0,len(filtered_sentence)):
        if(len(filtered_sentence[i])>2):# words of length 2 and less than 2 are of no use
            if filtered_sentence[i] in dictionary:
                dictionary[filtered_sentence[i]]=dictionary[filtered_sentence[i]]+1
            else:
                dictionary[filtered_sentence[i]]=1
            
    

In [ ]:
sorted_d = dict( sorted(dictionary.items(), key=operator.itemgetter(1),reverse=True))#sorting the dictionary on the basis of keys
#print(sorted_d)

In [ ]:
def top_k_words(dictionary):# returns the top k(=3000) words
    return sorted(dictionary, key=dictionary.get, reverse=True)[:2000]

In [ ]:
Final_dictionary=top_k_words(dictionary)
#print(Final_dictionary)
len(Final_dictionary)
index={}#creating another dictionary for further use which is just same as final_dictionary
j=0
for i in Final_dictionary:
    index[i]=j
    j=j+1
    

In [ ]:
print(len(xtrain))

In [ ]:
import numpy as np
print()
l1=np.zeros((len(xtrain),2000))# creating a 2d array of len(xtrain)*3000 features since features are 3000
#print(l1.shape)
index1=0
for i in xtrain:
    word_tokens = i[1].split()
    filtered_sentence = [w.lower() for w in word_tokens] 
    for j in range(0,len(filtered_sentence)):#filling of 2d array with the help of duplicate dictionary created earlier
        if(filtered_sentence[j] in index):
            l1[index1][index[filtered_sentence[j]]]= l1[index1][index[filtered_sentence[j]]]+1
    index1=index1+1
   
    
            

In [ ]:
def predicting(l1,ytrain,index):
    result={}
    all_classes=set(ytrain)
    #print(all_classes)
    result['total_values']=len(ytrain)
    #l1[0]
    for current_class in all_classes:
        sum1=0
        result[current_class]={}
        current_working_rows=l1[(ytrain==current_class)]
        #print(current_working_rows)
        result[current_class]['total_count']=len(current_working_rows)
        for i in index:
            #print(i)
            result[current_class][i]=current_working_rows[:,index[i]].sum()
        for i in index:
            sum1=sum1+result[current_class][i]
        result[current_class]['total_words']=sum1
    return result
        
        
        

In [ ]:
result=predicting(l1,ytrain,index)

In [ ]:
ytrain=[i for i in ytrain]
ytrain=np.array(ytrain)
ytrain

In [ ]:
def probability(index,dictionary,current,x):
    output=np.log(dictionary[current]['total_count'])-np.log(dictionary['total_values'])
    word_tokens = x.split()
    filtered_sentence = [w.lower() for w in word_tokens] 
    for i in range(0,len(filtered_sentence)):
        if(filtered_sentence[i] in index):
            count_values_with_value_xj=dictionary[current][filtered_sentence[i]]+1 #LAPLACE CORRECTION
            count_current_class=dictionary[current]['total_words']+len(index.keys())#LAPLACE CORRECTION
            output=(output)+np.log(count_values_with_value_xj)-np.log(count_current_class)
            
    return output

In [ ]:
def predict_single_point(index,dictionary,x):
    class_values=dictionary.keys()
    result=-1
    best_class=-11
    first_run=True
    for current in class_values:
        if(current=='total_values'):
            continue
        ans=probability(index,dictionary,current,x)
        if(first_run or ans>result):
            result=ans
            best_class=current
        first_run=False
    return best_class
    
    

In [ ]:
def predict(index,result,xtest):
    ypredictions=[]
    for x in xtest:
        
        answer=predict_single_point(index,result,x[1])
        ypredictions.append(answer)
    return ypredictions
    

In [ ]:
ypredictions=predict(index,result,xtest)


In [ ]:
ypredictions

In [ ]:
ypredictions=np.array(ypredictions)

In [ ]:
ytest=[i for i in ytest]
ytest=np.array(ytest)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(ytest,ypredictions))
print(confusion_matrix(ytest,ypredictions))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
#usinf inbuilt multimonial nb
clf=MultinomialNB()
clf.fit(xtrain,ytrain)
#y_sklearn_pred=clf.predict(xtest)
clf.score(xtest,ytest)